In [1]:

#^###############################  Storage_Optimization.ipynb  ####################################
# ^ Author: Sukhendu Sain
# ^ Description: Main Codebase to give estimate on Zoning and Bin Allocation based on Part Categorization
# ^ Data: 03-Mar-2025
#^################################################################################

In [2]:
# Import Necessary Libraries, Utils, and Config Files
from config import *
from tqdm import tqdm
from pypdf import PdfReader
import pandas as pd
import numpy as np
import math
import openpyxl
import utils

# Data Import and Clean

## Data Processing & Calculation

In [ ]:

# * SAVE the Marged Main Dataset into Excel, To Avoid The Above Steps During Rerun
df_Main.to_excel(r"D:\Sukhendu\UPWORK-WORK\0_Dondray_Auto\StorageRackOptimizationNew\Data&Files\Gparts-MergedMainData.xlsx", index=False) 
# ~8 Secs

In [170]:
#####   READ  ALREADY SAVED GPARTS-Merged Data
## @ Read FILE:: (Gparts-MergedMainData.xlsx) into Dataframe
df_Main = utils.read_excel(r"D:\Sukhendu\UPWORK-WORK\0_Dondray_Auto\StorageRackOptimizationNew\Data&Files\Gparts-MergedMainData.xlsx").fillna("")
df_Main['Part Desc.'] = df_Main['Part Desc.'].apply(lambda desc: "-".join(desc.split("-")[1:]))
# ~6 secs

In [ ]:

## @ Read FILE:: (GMPARTS Parts 2 xlsx Files into Dataframe
GMPARTS_FILE_PATH1 = r"D:\Sukhendu\UPWORK-WORK\0_Dondray_Auto\StorageRackOptimizationNew\Data&Files\All_Parts_for_BS__02_06_2025.xlsx"
GMPARTS_FILE_PATH2 = r"D:\Sukhendu\UPWORK-WORK\0_Dondray_Auto\StorageRackOptimizationNew\Data&Files\All_Parts_for_BS-3__02_06_2025.xlsx"

df_GMParts1 = utils.read_excel(GMPARTS_FILE_PATH1)
df_GMParts2 = utils.read_excel(GMPARTS_FILE_PATH2)

df_GMParts = pd.concat([df_GMParts1, df_GMParts2])
df_GMParts = df_GMParts[['GM NUMBER', 'PART DESCRIPTION', 'PACKAGE HEIGHT', 'PACKAGE WIDTH', 'PACKAGE LENGTH']]
df_GMParts.columns = ['Part#', 'Part Desc.', 'Height', 'Width', 'Length']
df_GMParts.insert(2, 'OH Inventory', 0)
df_GMParts.insert(2, 'Part Category', "")
df_GMParts.insert(4, '0Dimensions', False)
df_GMParts.insert(7, 'Zoning', "")
df_GMParts.insert(8, 'StorageType', "")
df_GMParts.insert(9, 'SubStorage', "")
df_GMParts.insert(10, 'Bin Type', "")
df_GMParts.insert(11, 'Num. Bin Required', 0)

df_GMParts.loc[(df_GMParts["Depth"] == 0) | (df_GMParts["Height"] == 0) | (df_GMParts["Width"] == 0), "0Dimensions"] = True

df_GMParts = df_GMParts.reset_index(drop=True)
df_GMParts["PART DESCRIPTION"] = df_GMParts["PART DESCRIPTION"].fillna("")

if print_df_after_import: utils.print_df(df_GMParts) # Print the Dataframe
# ~2 mins 38 secs

In [206]:
utils.print_df(df_GMParts, 1)

╒════╤══════════╤════════════════════════════╤═════════════════╤════════════════╤═══════════════╤══════════╤═════════╤═════════╤══════════╤═══════════════╤══════════════╤════════════╤═════════════════════╕
│    │    Part# │ Part Desc.                 │ Part Category   │   OH Inventory │ 0Dimensions   │   Height │   Width │   Depth │ Zoning   │ StorageType   │ SubStorage   │ Bin Type   │   Num. Bin Required │
╞════╪══════════╪════════════════════════════╪═════════════════╪════════════════╪═══════════════╪══════════╪═════════╪═════════╪══════════╪═══════════════╪══════════════╪════════════╪═════════════════════╡
│  0 │ 10000068 │ BUSHING,R/AXL UPR CONT ARM │ Chassis         │              0 │ False         │      2.6 │     2.6 │     3.2 │ Blue     │               │              │            │                   0 │
╘════╧══════════╧════════════════════════════╧═════════════════╧════════════════╧═══════════════╧══════════╧═════════╧═════════╧══════════╧═══════════════╧══════════════╧══════

## Part Categorization

In [171]:
def part_categorization(df_toBeCategorized, categorizationRules, categoryColName, categoryDescName):
    
    # Apply categorization
    def categorizeGM(desc):
        desc = desc.lower().strip()       
        category = "Miscellaneous"
        
        for cat in categorizationRules:
            if cat[0] == 'equal' and cat[1] == desc:
                category = cat[2]
                break
            elif cat[0] == 'include' and cat[1] in desc:
                category = cat[2]
                break
            elif cat[0] == 'include2' and cat[1] in desc and cat[2] in desc:
                category = cat[3]
                break
            elif cat[0] == 'include3' and cat[1] in desc and cat[2] in desc and cat[3] in desc:
                category = cat[4]
                break
        
        return category
        
    df_toBeCategorized[categoryColName] = df_toBeCategorized[categoryDescName].astype(str).apply(categorizeGM)

    

In [172]:
part_categorization(df_Main, categorizationRulesGM, 'Part Category 2', 'Part Desc.')
part_categorization(df_GMParts, categorizationRulesGM, 'Part Category', 'Part Desc.')

In [173]:
df_GMParts[df_GMParts["Part Category"] != "Miscellaneous"].shape[0], df_GMParts[df_GMParts["Part Category"] == "Miscellaneous"].shape[0]

(1469701, 13043)

In [174]:
df_Main[df_Main["Part Category 2"] != "Miscellaneous"].shape[0], df_Main[df_Main["Part Category 2"] == "Miscellaneous"].shape[0]

(58960, 3975)

In [175]:
testy = pd.DataFrame(zip(df_Main.groupby('Part Category 2')['Total Sold'].sum().index, df_Main.groupby('Part Category 2')['Total Sold'].sum().values, df_Main['Part Category 2'].value_counts()))
testy[3] = testy.iloc[:, 1].div(testy.iloc[:, 2], axis=0)
testy[4] = testy.iloc[:, 1].mul(testy.iloc[:, 3], axis=0)
testy[5] = testy.iloc[:, 1].div(testy.iloc[:, 1].sum())
testy.insert(6, 'Zone', "")
testy.columns = ['Category', 'Total Sold', 'Count', 'Average Sold', '(Total * Average) Sold', 'Total Sold %', 'Zone']
testy.loc[testy["Category"] == "Oil", "Total Sold"] = 0
testy

,Category,Total Sold,Count,Average Sold,(Total * Average) Sold,Total Sold %,Zone
0,Abs,742,3975,0.186667,1.385067e+02,0.001871,
1,Accumulator,51,3018,0.016899,8.618290e-01,0.000129,
2,Actuators,239,2558,0.093432,2.233034e+01,0.000603,
3,Adapters,0,1881,0.000000,0.000000e+00,0.000000,
4,Air Cleaner,3759,1578,2.382129,8.954424e+03,0.009479,
...,...,...,...,...,...,...,...
263,Windows,169,1,169.000000,2.856100e+04,0.000426,
264,Windshield,2,1,2.000000,4.000000e+00,0.000005,
265,Wiper,6454,1,6454.000000,4.165412e+07,0.016276,
266,Wire,1669,1,1669.000000,2.785561e+06,0.004209,


## Apply Zoning

In [176]:
totalDaysOfData = 300    # Total Days in Dataset
redHot1SaleTP = 1        # 1 Sell per 1 day
red1SaleTP =  7          # 1 sell per 7 days (1 week)   
orange1SaleTP = 14       # 1 sell per 14 days  (2 weeks) 
yellow1SaleTP = 21       # 1 sell per 21 days  (3 weeks)
green1SaleTP = 30        # 1 sell per 30 days  (1 month)
blue1SaleTP = float("inf")     # 1 sell per 300 Days (~Rest of Data)

zones = {
    'Red Hot': totalDaysOfData/redHot1SaleTP,
    'Red': totalDaysOfData/red1SaleTP,
    'Orange': totalDaysOfData/orange1SaleTP,
    'Yellow': totalDaysOfData/yellow1SaleTP,
    'Green': totalDaysOfData/green1SaleTP,
    'Blue': totalDaysOfData/blue1SaleTP
}

In [177]:

# @ Apply Zoning based on Time Period/Sale

## * Main Function for Apply Zoning
def Apply_Zoning(df_toBeZoned, zones, soldColName='Total Sold', zoneColName='Zone'): 
    df_toBeZoned.loc[:, zoneColName] = df_toBeZoned[soldColName].apply(lambda x: next((zone for zone, ratio in zones.items() if x >= ratio), list(zones.keys())[-1]))
    df_toBeZoned.loc[df_toBeZoned[soldColName] < 0, zoneColName] = None

In [139]:
stored_soldPercents = []
zones = ['Red Hot', 'Red', 'Orange', 'Yellow', 'Green', 'Blue']
zi = 0
li = 0
for len in range(testy.shape[0]):
    if sum(stored_soldPercents) >= 0.20:
        testy.loc[li:len, 'Zone'] = zones[zi]
        li = len
        zi += 1
        stored_soldPercents = []
    stored_soldPercents.append(testy.loc[len, 'Total Sold %'])

In [178]:
for len in range(testy.shape[0]):
    print(testy.iloc[len])
    if testy.loc[len, 'Total Sold %'] > 0.24:
        testy.loc[len, 'Zone'] = 'Red Hot'
    elif testy.loc[len, 'Total Sold %'] > 0.06:
        testy.loc[len, 'Zone'] = 'Red'
    elif testy.loc[len, 'Total Sold %'] > 0.015:
        testy.loc[len, 'Zone'] = 'Orange'
    elif testy.loc[len, 'Total Sold %'] > 0.005:
        testy.loc[len, 'Zone'] = 'Yellow'
    elif testy.loc[len, 'Total Sold %'] > 0.001:
        testy.loc[len, 'Zone'] = 'Green'
    else:
        testy.loc[len, 'Zone'] = 'Blue'

Category                         Abs
Total Sold                       742
Count                           3975
Average Sold                0.186667
(Total * Average) Sold    138.506667
Total Sold %                0.001871
Zone                                
Name: 0, dtype: object
Category                  Accumulator
Total Sold                         51
Count                            3018
Average Sold                 0.016899
(Total * Average) Sold       0.861829
Total Sold %                 0.000129
Zone                                 
Name: 1, dtype: object
Category                  Actuators
Total Sold                      239
Count                          2558
Average Sold               0.093432
(Total * Average) Sold    22.330336
Total Sold %               0.000603
Zone                               
Name: 2, dtype: object
Category                  Adapters
Total Sold                       0
Count                         1881
Average Sold                   0.0
(Total * Avera

In [ ]:

## * Run the Apply_Zoning on df_Main
Apply_Zoning(testy, zones, 1, 4)

In [179]:
testy['Zone'].value_counts()

Zone
Blue       189
Green       50
Yellow      16
Orange       9
Red          3
Red Hot      1
Name: count, dtype: int64

In [261]:
utils.print_df(testy.sort_values('Total Sold', ascending=False))

╒═════╤════════════════════════╤══════════════╤═════════╤════════════════╤══════════════════════════╤════════════════╤═════════╕
│     │ Category               │   Total Sold │   Count │   Average Sold │   (Total * Average) Sold │   Total Sold % │ Zone    │
╞═════╪════════════════════════╪══════════════╪═════════╪════════════════╪══════════════════════════╪════════════════╪═════════╡
│ 156 │ Motorcraft             │        96857 │      46 │  2105.59       │              2.03941e+08 │    0.244253    │ Red Hot │
├─────┼────────────────────────┼──────────────┼─────────┼────────────────┼──────────────────────────┼────────────────┼─────────┤
│  27 │ Bolt                   │        29012 │     641 │    45.2605     │              1.3131e+06  │    0.0731621   │ Red     │
├─────┼────────────────────────┼──────────────┼─────────┼────────────────┼──────────────────────────┼────────────────┼─────────┤
│  93 │ Filters                │        26390 │     131 │   201.45       │              5.31628e+

In [181]:
df_GMParts["Zoning"] = df_GMParts["Part Category"].map(testy.set_index('Category')['Zone'])

In [245]:

for cat in set(df_GMParts['Part Category'].unique()):
    if cat not in set(df_Main['Part Category 2'].unique()):
        print(cat)

Media
Connector Harn
Price List (comm)
Head Speedometer
Manual
Balancer
Catalog
Eprom
Differential Bearing
Theft Transmitter
Navigation
Yoke Slip
Sph Shield
Air Cooler
Control Transmission
Shield Engine
Exhaust
Fuel Meter
Sidedoor Moulding
Side Door Pawl
Speedometer
Emergency Flasher
Cab Shafts
Decor
Adjustment Kit
Fascia
Twc Converter
Hoist
Parking Switch
Detent
Brush
Buckle
Cover
Moulding Fascia
Calibrator
Check Side Door
Tubing
Attachment Kit
Hazard Flasher
Info Center Telltale
Conversion Valve
Thermostat Engine
Tanks
Radiator Core
Wheel Shield
Guide Valve Lifter
Restraint
Hub Asy
Floor Console Stripe
Carburetor
Oil Seal
Grille Asy
Cabin
Drag Link
Platform
Transmission Indicator
Wrench
Trans Fluid
Check Link
Luggage Shade
Ring Piston
Push Button
Mount Eng
PCV
Yoke Differential
Starter Asy
Dampener
Fender Moulding
Cleaner
Radiator Tank
Instrumentation
Shutter
Fuel Gauge
Dryer
Input Shaft Ring Bearing
Harness Asy
Yoke Propeller
Installation Kit
Lock Asy
Fluid
Shield Noise
Battery Cell

### Specialty Storage Assignment

#### Function for Bin Calculation

In [183]:
def getNumOfBin(depth, width, height, raw_bin_dim, ohInven, fillFactor):
    # * Raw Bin Dimensions has this format :-  Height_Depth_Width
    if (raw_bin_dim != "") and (ohInven > 0):
        bin_height = float(raw_bin_dim.split("_")[1])
        bin_depth = float(raw_bin_dim.split("_")[2])
        bin_width = float(raw_bin_dim.split("_")[3])

        if raw_bin_dim.split("_")[0] == "BR":   # * Battery Rack
            return round((ohInven * width) / bin_depth, 3)      
               
        volBin = fillFactor * bin_height * bin_depth * bin_width    # * Available Storage Space
        volPart = height * depth * width
        if (volBin == 0):
            return 0
        
        numOfBins = round((ohInven * volPart) / volBin, 3)      # * Returns Fraction. 
        return numOfBins
    else:
        return 0

### Main Function for Storage Assignment

In [233]:
def getStorage(zone, pcate, depth, width, height, ohInven, fillFactor):
    # Initialize the empty Variables
    storageType = ""
    subStorage = ""

    if zone == "":
        return storageType, subStorage, "", 0 # Return the Values 

    isSpec, storageType, subStorage, raw_bin_dim = utils.getSpecialtyStorage(pcate, depth, width, height)

    if not isSpec: 
        if (zone == "Red Hot") | (zone == "Red"):
            storageType, subStorage, raw_bin_dim = utils.getRedHotStorage(depth, width, height)
        elif (zone == "Orange") | (zone == "Yellow"):
            storageType, subStorage, raw_bin_dim = utils.getOrangeYellowStorage(depth, width, height)
        elif (zone == "Green") | (zone == "Blue"):   
            storageType, subStorage, raw_bin_dim = utils.getGreenBlueStorage(depth, width, height)
 
    #numOfBins = getNumOfBin(depth, width, height, raw_bin_dim, ohInven, fillFactor)
    binDim = ""

    # * Build Bin Label with C (Clip), B (Bulk), D (Drawer), Battery Rack (BR), Tire Rack (TR) and Width-Depth-Height
    if raw_bin_dim.strip():   
        binDim =  raw_bin_dim.split('_')[0] + raw_bin_dim.split('_')[3] + raw_bin_dim.split('_')[2] + raw_bin_dim.split('_')[1]
    

    return storageType, subStorage, binDim, 0 # Return the Values

#### Apply the Storage Function

In [237]:
# Precompute masks and initialize columns properly
new_columns = ["StorageType", "SubStorage", "Bin Type", "Num. Bin Required"]
zero_dim_mask = df_GMParts["0Dimensions"]

# Initialize all new columns with proper data types for all rows
# for col in new_columns:
#     if col not in df_GMParts:
#         df_GMParts[col] = None  # Create as object dtype

# Handle zero dimensions case (set empty values for all columns)
df_GMParts.loc[zero_dim_mask, new_columns] = ("", "", "", 0)

# Process valid dimensions using vectorized operations
valid_mask = ~zero_dim_mask
tqdm.pandas(desc="Processing")

# Apply getStorage to valid rows and update columns
def process_row(row):
    return getStorage(
        row["Zoning"],
        row["Part Category"],
        row["Depth"],
        row["Width"],
        row["Height"],
        row["OH Inventory"],
        fillFactor
    )

df_GMParts.loc[valid_mask, new_columns] = (
    df_GMParts[valid_mask]
    .progress_apply(process_row, axis=1, result_type='expand')
    .values  # Use .values to bypass index alignment issues
)

Processing: 100%|██████████| 1001476/1001476 [00:26<00:00, 38363.37it/s]


In [273]:
df_GMParts['Zoning'].value_counts()

Zoning
Blue      455547
Green     315080
Yellow    167757
Orange    144829
Red        35582
Name: count, dtype: int64

In [272]:
utils.print_df(df_GMParts[df_GMParts['0Dimensions'] == False][df_GMParts['Zoning'] == 'Orange'][:100])

╒══════╤══════════╤════════════════════════════════════════════════╤═════════════════╤════════════════╤═══════════════╤══════════╤═════════╤═════════╤══════════╤═══════════════════════════╤═══════════════════════════════════╤════════════╤═════════════════════╕
│      │    Part# │ Part Desc.                                     │ Part Category   │   OH Inventory │ 0Dimensions   │   Height │   Width │   Depth │ Zoning   │ StorageType               │ SubStorage                        │ Bin Type   │   Num. Bin Required │
╞══════╪══════════╪════════════════════════════════════════════════╪═════════════════╪════════════════╪═══════════════╪══════════╪═════════╪═════════╪══════════╪═══════════════════════════╪═══════════════════════════════════╪════════════╪═════════════════════╡
│   17 │ 10000812 │ BUMPER-R/AXL                                   │ Body            │              0 │ False         │     2.7  │    2.1  │    2    │ Orange   │ Clip Shelving             │ 12-inch Deep - 36-inch Wide

C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_9084\1685366731.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  utils.print_df(df_GMParts[df_GMParts['0Dimensions'] == False][df_GMParts['Zoning'] == 'Orange'][:100])


In [ ]:
for i in tqdm(range(df_GMParts.shape[0]), desc="Completion"):
#for i in range(df_Main.shape[0]):
    # Set the Dimensions of the Data into Variables
    depth = df_GMParts.loc[i, "Depth"]
    height = df_GMParts.loc[i, "Height"]
    width = df_GMParts.loc[i, "Width"]

    zone = df_GMParts.loc[i, "Zoning"]
    pcate = df_GMParts.loc[i, "Part Category"]
    ohInven = df_GMParts.loc[i, "OH Inventory"]

    # * If any dimension is zero, set empty Storage
    if df_GMParts.loc[i, "0Dimensions"] == True:
        df_GMParts.loc[i, "StorageType"] = ""
        df_GMParts.loc[i, "SubStorage"] = ""
        continue

    # Set Storage of the Parts
    df_GMParts.loc[i, "StorageType"], df_GMParts.loc[i, "SubStorage"], df_GMParts.loc[i, "Bin Type"], df_GMParts.loc[i, "Num. Bin Required"] = getStorage(zone, pcate, depth, width, height, ohInven, fillFactor)

# ~34 secs

In [ ]:

## @ HANGING Storage Calculation
# TODO: To Get the SKU Count for Hanging Storage
# * ASSUMPTION: Hook Length Based on SKU Count:
# *             6-inch hooks: For SKUs with 10 items or fewer
# *             12-inch hooks: For SKUs with 10–20 items

for hangingPN in tqdm(df_Main.loc[df_Main['StorageType'] == 'Hanging Specialty Storage', "Part#"], "Completion"): # Get Hanging Parts
    # & df_Main.loc[(df_Main['Part#'] == hangingPN), "Num. Bin Required"] = round(int(df_Main.loc[(df_Main['Part#'] == hangingPN), "OH Inventory"].values[0]) / hookDiv, 4)
    df_Main.loc[(df_Main['Part#'] == hangingPN), "SubStorage"], df_Main.loc[(df_Main['Part#'] == hangingPN), "Bin Type"], hookDiv = ("6-inch Hook", "HS06", 10) if df_Main.loc[(df_Main['Part#'] == hangingPN), "SKU Count"].values[0] <= 10 else ("12-inch Hook", "HS12", 20)
    df_Main.loc[(df_Main['Part#'] == hangingPN), "Num. Bin Required"] = int(df_Main.loc[(df_Main['Part#'] == hangingPN), "OH Inventory"].values[0])  # * Set No. of Hooks = Inventory Count

###  SET SKU Count to zero for all other STORAGE Types 
df_Main.loc[df_Main['StorageType'] != 'Hanging Specialty Storage', "SKU Count"] = 0

# ~46 secs

In [ ]:

## @ Tire Storage Calculation
# TODO: FINALIZE  the Calculation of Tire Carousel Model Selection Based on Percentage 
# * ASSUMPTION:  Assign Carousel Model based on Diameter Group %age
# *   If 50% of Tires Have 33-inch or More Diameter, Assign Large-Storage (72-width carousel)
# *   ELSE,  For 28-inch or less, and,  between 28-33 inches, assign standard carrousel (48-width carousel) 
carousel_model = ''
if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] > 0: 
    carousel_model = 'TR72' if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '33-inches Dia'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= tirePercent else 'TR48'
    carousel_width = 72 if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '33-inches Dia'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= tirePercent else 48
    for tirePN in df_Main.loc[df_Main['StorageType'] == 'Tire Specialty Storage', "Part#"]:
        df_Main.loc[(df_Main['Part#'] == tirePN), "Num. Bin Required"] = round(int(df_Main.loc[(df_Main['Part#'] == tirePN), "OH Inventory"].values[0]) / (carousel_width // int(df_Main.loc[(df_Main['Part#'] == tirePN), "Width"].values[0])), 3)
        df_Main.loc[(df_Main['Part#'] == tirePN), "Bin Type"] = carousel_model

# ~0.3 secs

In [ ]:

# @ Do Actual Storage Allocation based on Inventory, Number of Bins availiable, Handle Overflow, etc...

df_binData = pd.DataFrame(columns=['Bin Label', 'Bin Category', 'Total Bins', 'Filled Amount', 'Bin Order', 'Bin Location', 'Availiability Flag'])

# * High-Density Drawers (2)
binData = [ 
    {'Bin Label': 'D362406', 'Bin Category': 'Drawer', 'Total Bins': 5, 'Filled Amount': 0, 'Bin Order': 1, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'D482406', 'Bin Category': 'Drawer', 'Total Bins': 4, 'Filled Amount': 0, 'Bin Order': 2, 'Bin Location': 'None', 'Availiability Flag': 'Yes'}
 ]
# * Clip-Shelving (6)
binData.extend([
    {'Bin Label': 'C361215', 'Bin Category': 'Clip', 'Total Bins': 4, 'Filled Amount': 0, 'Bin Order': 3, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'C361815', 'Bin Category': 'Clip', 'Total Bins': 6, 'Filled Amount': 0, 'Bin Order': 4, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'C362415', 'Bin Category': 'Clip', 'Total Bins': 8, 'Filled Amount': 0.0, 'Bin Order': 5, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},    
    {'Bin Label': 'C481215', 'Bin Category': 'Clip', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 6, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},  
    {'Bin Label': 'C481815', 'Bin Category': 'Clip', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 7, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},   
    {'Bin Label': 'C482415', 'Bin Category': 'Clip', 'Total Bins': 7, 'Filled Amount': 0.0, 'Bin Order': 8, 'Bin Location': 'None', 'Availiability Flag': 'Yes'}
])

# * Bulk-Storage (18)
binData.extend([
  {'Bin Label': 'B482448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 9, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B483648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 10, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B484248', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 11, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B484848', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 12, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B487248', 'Bin Category': 'Bulk', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 13, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B489648', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 14, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B722448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 15, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B723648', 'Bin Category': 'Bulk', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 16, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B724248', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 17, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B724848', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 18, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B727248', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 19, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B729648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 20, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B962448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 21, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B963648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 22, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B964248', 'Bin Category': 'Bulk', 'Total Bins': 7, 'Filled Amount': 0.0, 'Bin Order': 23, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B964848', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 24, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B967248', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 25, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B969648', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 26, 'Bin Location': 'None', 'Availiability Flag': 'Yes'}
])

# * Specialty (7) TR
binData.extend([
    {'Bin Label': 'BR484816', 'Bin Category': 'Battery', 'Total Bins': 14, 'Filled Amount': 0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'TR48', 'Bin Category': 'Tire', 'Total Bins': 6, 'Filled Amount': 0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': "TR72", 'Bin Category': 'Tire', 'Total Bins': 6, 'Filled Amount': 0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'BC967248', 'Bin Category': 'Bumper Cover', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},  
    {'Bin Label': 'BH967280', 'Bin Category': 'Hood', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},   
    {'Bin Label': 'HS06', 'Bin Category': 'Hanging', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},   
    {'Bin Label': 'HS12', 'Bin Category': 'Hanging', 'Total Bins': 7, 'Filled Amount': 0.0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'}
])

# Append the Data to the DF
df_binData = pd.concat([df_binData, pd.DataFrame(binData)], ignore_index=True)
df_binData['Total Bins'] = np.random.choice(range(40, 400), df_binData.shape[0])

In [ ]:
utils.print_df(df_binData)

In [ ]:
# * Write  the Final BINS Dataset into Excel
#df_binData.to_excel('All_Bins_Data.xlsx', index=False) 

In [ ]:

## ^ Psuedocode for main Logic
# ~ Start with 'Bin Type' as default assignment for the part --  ALLOCATE Bin As Per Availiability, Zone, and Priority Rules
# ~ Add a column to 'BINS' dataframe, as Order or priority based on dimension for a given Rack Type (Drawer, Bulk, Clip, etc). 
# ~  This Order should only within specific Rack Type (Drawer, Bulk, Clip, etc). To change to Rack Type, use priorities given for 
# ~ Zones and Part Category. Always start with minimum dimension of the BIN
# ~ Start with 'Red Hot' Zone Parts to allocate Actual Bin. 
# ~ LOOP Over Zones. Then LOOP Over Parts. For Each Part, Check default 'Bin Type'. If Available for storage, then allocate it  
# ~ (Add the Bin Type in 'Actual Bin Allocation',  Increment the Fill_Quantity for the 'Bin Type' in second dataframe for BINS. 
# ~ Fill_Quantity can be fraction.  If Fill_Quantity is same  as  Available Bins, then Set the 'Available FLAG' for the BIN to 'NO'
# ~ That means, the BIN is fully occupied. Pick the next available BIN. 
# ~ Think About Overflow,  and Bin Location (Not sure if Bin Locations will be zone-wise)

In [ ]:
####  BIN  ALLOCATION
## @ Red Hot, Red, Orange, and Yellow Zone Non-Specialty Parts
for pn in tqdm(df_Main.loc[(df_Main['Zone'] == 'Red Hot') | (df_Main['Zone'] == 'Red') | (df_Main['Zone'] == 'Orange') | (df_Main['Zone'] == 'Yellow'), 'Part#'], "Completion"):
    # Get & Set Variables
    actualBin =  ""
    overflowBin = ""
    overflowComment = ""

    partData = df_Main[df_Main['Part#'] == pn]
    partHeight = partData['Height'].values[0]
    partWidth = partData['Width'].values[0]
    partDepth = partData['Depth'].values[0]
    partVolume = partHeight * partWidth * partDepth 
    binType = partData['Bin Type'].values[0]
    storageType = partData['StorageType'].values[0]
    partOHInven = partData['OH Inventory'].values[0]


    # ~ Base Continue Case - If no Storage Assignment, no Inventory Parts or is Specialty Part
    if (binType == "") or ("Specialty Storage" in storageType) or (partOHInven <= 0):     
        continue

    binData = df_binData.loc[df_binData['Bin Label'] == binType]
    totalBinOfType = binData['Total Bins'].values[0]
    filledAmtOfBin = binData['Filled Amount'].values[0]
    binOrder = binData['Bin Order'].values[0]
    flagAvail = binData['Availiability Flag'].values[0]

    # Calculate Variables
    binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
    remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
    partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

    if partsAllocated > 0:   # * If Actual Bin is Availiable
        # Calculate Variables
        totalPartVolume = partOHInven * partVolume
        actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
        numBins = round(totalPartVolume / binVolume, 3)

        # Set Values
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
        actualBin = f"{binType} ({actualBinRequired}, {partsAllocated})"

        if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): # If Bin is FULL, Reset Flag
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

        if (filledAmtOfBin + numBins) > (totalBinOfType- 0.01):   # * If Overflow (Actual Bin can't fit all Parts) (Only 1)
            overflowParts = partOHInven - partsAllocated
            
            # Find next availiable BIN to  handle overflow parts
            binFound = False
            for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
                # Calculate Variables
                binData = df_binData[df_binData['Bin Label'] == binType1]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]
                binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
                remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
                partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

                if partsAllocated > 0:   # If Actual Bin is Availiable
                    binFound = True
                    break
                #  OLD Code
                # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
                #     binData = df_binData[df_binData['Bin Label'] == binType1]
                #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
                #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
                #     if (fillAmt < (totalBin - 0.01)):
                #         break

            if not (binFound):
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part1- No More Available/Fitting Bins"
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
                continue

            # Get & Calculate Variables
            binType = binData['Bin Label'].values[0]
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
            actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)

            # Add  FilledAmount for OverFlow Bin  
            df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
            overflowBin = f"{binType} ({actualBinRequired}, {partsAllocated})"
        
            # If more overflow
            leftParts = overflowParts - partsAllocated
            if leftParts > 0:
                binsNeeded = round(leftParts * partVolume / binVolume, 3)
                overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"
            
            if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): 
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    else:    # * If Actual Bin is not avaialble
        binFound = False
        for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
            # Calculate Variables
            binData = df_binData[df_binData['Bin Label'] == binType1]
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

            if partsAllocated > 0:
                binFound = True
                break
            #  OLD Code
            # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
            #     binData = df_binData[df_binData['Bin Label'] == binType1]
            #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
            #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
            #     if (fillAmt < (totalBin - 0.01)):
            #         break

        if not (binFound):
            df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = "Part2- No More Available/Fitting Bins"
            df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = "Part2- No More Available/Fitting Bins"
            df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{partOHInven} Parts Left; No Bins availiable to fit"
            continue

        binType = binData['Bin Label'].values[0]
        totalBinOfType = binData['Total Bins'].values[0]
        filledAmtOfBin = binData['Filled Amount'].values[0]
        binOrder = binData['Bin Order'].values[0]
        binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
        remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume   ## Check for Remaining Vol in Bin 
        partVolume = partHeight * partWidth * partDepth 
        totalPartVolume = partOHInven * partVolume
        numBins = round(totalPartVolume / binVolume, 3)         ### Number of Bins Required to fill Inventry Parts
        partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)
        actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
        
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
        actualBin = f"{binType} ({actualBinRequired}, {partsAllocated})"

        if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): 
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

        if filledAmtOfBin + numBins > totalBinOfType:   # * If Overflow (Actual Bin can't fit all Parts) (Only 1)
            overflowParts = partOHInven - partsAllocated
            for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
                # Calculate Variables
                binData = df_binData[df_binData['Bin Label'] == binType1]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]
                binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
                remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
                partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

                if partsAllocated > 0:
                    binFound = True
                    break
                #  OLD Code
                # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
                #     binData = df_binData[df_binData['Bin Label'] == binType1]
                #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
                #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
                #     if (fillAmt < (totalBin - 0.01)):
                #         break

            if not (binFound):
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part3- No More Available/Fitting Bins"
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
                continue

            binType = binData['Bin Label'].values[0]
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
            actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
            
            df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
            overflowBin = f"{binType} ({actualBinRequired}, {partsAllocated})"
        
            leftParts = overflowParts - partsAllocated
            if leftParts > 0:
                binsNeeded = round(leftParts * partVolume / binVolume, 2)
                overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"

            if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): 
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = actualBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = overflowBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = overflowComment

# ~1 min 15 secs

In [ ]:

## @ Green and Blue Zone Non-Specialty Parts
for pn in tqdm(df_Main.loc[(df_Main['Zone'] == 'Green') | (df_Main['Zone'] == 'Blue'), 'Part#'], "Completion"):
    # Get Variable
    actualBin =  ""
    overflowBin = ""
    overflowComment = ""

    partData = df_Main[df_Main['Part#'] == pn]
    partHeight = partData['Height'].values[0]
    partWidth = partData['Width'].values[0]
    partDepth = partData['Depth'].values[0]
    binType = partData['Bin Type'].values[0]
    storageType = partData['StorageType'].values[0]
    partOHInven = partData['OH Inventory'].values[0]

    ## * If Inventry Qty is zero, Go to Next Part -- Handle Specialty Storage separately 
    if (binType == "") or ("Specialty Storage" in storageType) or (partOHInven <= 0):     
        continue

    binData = df_binData.loc[df_binData['Bin Label'] == binType]
    totalBinOfType = binData['Total Bins'].values[0]
    filledAmtOfBin = binData['Filled Amount'].values[0]
    binOrder = binData['Bin Order'].values[0]
    partVolume = partHeight * partWidth * partDepth
 
    # Calculate Variables
    binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
    remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
    partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

    if partsAllocated > 0: # & (binData['Availiability Flag'].values[0] == "Yes"):    # If Actual Bin is Availiable
        binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
        remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
        partVolume = partHeight * partWidth * partDepth 
        totalPartVolume = partOHInven * partVolume
        numBins = round(totalPartVolume / binVolume, 3)
        partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)
        binRequired = round((partsAllocated * partVolume) / binVolume, 3)    

        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += binRequired
        actualBin = f"{binType} ({binRequired}, {partsAllocated})"

        if filledAmtOfBin + ((partsAllocated * partVolume) / binVolume) >= (totalBinOfType - 0.01):    ### @ ACtual BIN
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

        ###  Handle OVERFLOW  condition (Only 1 Overflow) 
        if filledAmtOfBin + numBins > totalBinOfType:
            
            overflowParts = partOHInven - partsAllocated
            for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
                # Calculate Variables
                binData = df_binData[df_binData['Bin Label'] == binType1]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]
                binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
                remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
                partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

                if partsAllocated > 0:   # @ If Actual Bin is Availiable
                    binFound = True
                    break
                #  OLD Code
                # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
                #     binData = df_binData[df_binData['Bin Label'] == binType1]
                #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
                #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
                #     if (fillAmt < (totalBin - 0.01)):
                #         break

            if not (binFound):
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part1-No More Available/Fitting Bins"
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
                continue

            binType = binData['Bin Label'].values[0]
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
            binRequired = round((partsAllocated * partVolume) / binVolume, 3)  

            overflowBin = f"{binType} ({binRequired}, {partsAllocated})"
        
            leftParts = overflowParts - partsAllocated
            if leftParts > 0:
                binsNeeded = round(leftParts * partVolume / binVolume, 3)
                overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"
            
            if (filledAmtOfBin + binRequired) >= (totalBinOfType - 0.01): 
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    else:    ## ^  If  suggested Bin Is NOT Avaialble.  Pick Next Available Bin and Process
        for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
            # Calculate Variables
            binData = df_binData[df_binData['Bin Label'] == binType1]
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

            if partsAllocated > 0:   # @ If Actual Bin is Availiable
                binFound = True
                break
            #  OLD Code
            # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
            #     binData = df_binData[df_binData['Bin Label'] == binType1]
            #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
            #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
            #     if (fillAmt < (totalBin - 0.01)):
            #         break

        if not (binFound):            
            df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = "Part2-No More Available/Fitting Bins"
            df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = "Part2-No More Available/Fitting Bins"
            df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{partOHInven} Parts Left; No Bins availiable to fit"
            continue

        binType = binData['Bin Label'].values[0]
        totalBinOfType = binData['Total Bins'].values[0]
        filledAmtOfBin = binData['Filled Amount'].values[0]
        binOrder = binData['Bin Order'].values[0]
        binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
        remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume   ## Check for Remaining Vol in Bin 
        partVolume = partHeight * partWidth * partDepth 
        totalPartVolume = partOHInven * partVolume
        numBins = round(totalPartVolume / binVolume, 3)         ### Number of Bins Required to fill Inventry Parts
        partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)
        binRequired = round((partsAllocated * partVolume) / binVolume, 3)  
        
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += binRequired
        actualBin = f"{binType} ({binRequired}, {partsAllocated})"

        if (filledAmtOfBin + binRequired) >= (totalBinOfType - 0.01):
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

        ###  Handle OVERFLOW  condition (Only 1 Overflow) 
        if filledAmtOfBin + numBins > totalBinOfType:
            overflowParts = partOHInven - partsAllocated
            for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
                # Calculate Variables
                binData = df_binData[df_binData['Bin Label'] == binType1]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]
                binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
                remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
                partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

                if partsAllocated > 0:   # @ If Actual Bin is Availiable
                    binFound = True
                    break
                #  OLD Code
                # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
                #     binData = df_binData[df_binData['Bin Label'] == binType1]
                #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
                #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
                #     if (fillAmt < (totalBin - 0.01)):
                #         break

            if not (binFound):
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part3-No More Available/Fitting Bins"
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
                continue

            binType = binData['Bin Label'].values[0]
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
            binRequired = round((partsAllocated * partVolume) / binVolume, 3) 

            ## @ ADD Below Filled Amount for Overflow bins
            df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += binRequired
            overflowBin = f"{binType} ({binRequired}, {partsAllocated})"
        
            leftParts = overflowParts - partsAllocated
            if leftParts > 0:
                binsNeeded = round(leftParts * partVolume / binVolume, 3)
                overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"

            if (filledAmtOfBin + binRequired) >= (totalBinOfType - 0.01):
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = actualBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = overflowBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = overflowComment

    
# ~16 mins 12 secs

In [ ]:

## @ All Zone Specialty Parts
for pn in df_Main['Part#']:
    # Get & Set Variables
    actualBin =  ""
    overflowBin = ""
    overflowComment = ""

    partData = df_Main[df_Main['Part#'] == pn]
    partHeight = partData['Height'].values[0]
    partWidth = partData['Width'].values[0]
    partDepth = partData['Depth'].values[0]
    binType = partData['Bin Type'].values[0]
    partOHInven = partData['OH Inventory'].values[0]
    partSKUCount = partData['SKU Count'].values[0]

    # ~ Base Continue Case - If no Storage Assignment, no Inventory Parts or is not Specialty Part
    if (binType == "") or (partOHInven <= 0):     
        continue
    if (all([binLabelTypes not in binType.lower() for binLabelTypes in ['br', 'tr', 'hs', 'bc', 'bh']])):     
        continue

    binData = df_binData.loc[df_binData['Bin Label'] == binType]
    totalBinOfType = binData['Total Bins'].values[0]
    filledAmtOfBin = binData['Filled Amount'].values[0]

    ## * Calculation for Battery
    if "br" in binType.lower():
        remainingBinWidth = (totalBinOfType - filledAmtOfBin) * float(binType[4:6])
        partsAllocated = min(math.floor(remainingBinWidth / partWidth), partOHInven)
        binsRequired = round((partsAllocated * partWidth) / float(binType[4:6]), 4)
        partsLeft = partOHInven - partsAllocated
        overflowBinsNeeded = partsLeft / float(binType[2:4])
    ## * Calculation for Tire
    if "tr" in binType.lower():
        remainingBinWidth = (totalBinOfType - filledAmtOfBin) * float(binType[2:4])
        partsAllocated = min(math.floor(remainingBinWidth / partWidth), partOHInven)
        binsRequired = round((partsAllocated * partWidth) / float(binType[2:4]), 4)
        partsLeft = partOHInven - partsAllocated
        overflowBinsNeeded = round(partsLeft / float(binType[2:4]), 3)
    ## * Calculation for Hanging Storage
    if "hs" in binType.lower():
        partsAllocated = min(math.floor(totalBinOfType - filledAmtOfBin), partOHInven)
        binsRequired = round(partsAllocated, 4)
        partsLeft = partOHInven - partsAllocated
        overflowBinsNeeded = partsLeft
    ## * Calculation for Bumper Cover & Hoods
    if ("bc" in binType.lower()) | ("bh" in binType.lower()):
        partVolume = partHeight * partWidth * partDepth 
        binVol = fillFactor * (float(binType[2:4]) * float(binType[4:6]) * float(binType[6:8]))
        partsAllocated = min(math.floor(((totalBinOfType - filledAmtOfBin) * binVol) / partVolume), partOHInven)
        binsRequired = round((partsAllocated * partVolume) / binVol, 4)
        partsLeft = partOHInven - partsAllocated
        overflowBinsNeeded = round((partsLeft * partVolume) / binVolume, 3)

    ## * Main Data Update
    if filledAmtOfBin + binsRequired >= (totalBinOfType - 0.01): # Update Availiability Flag, if full
        df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    if partsAllocated > 0: # If any parts allocated
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += binsRequired
        actualBin = f"{binType} ({binsRequired}, {partsAllocated})"
    else:
        actualBin = "SP- No More Available/Fitting Bins" # If no parts can be allocated

    if partsLeft > 0: # If Overflow
        overflowBin = "SP- No More Available/Fitting Bins"
        overflowComment = f"{partOHInven - partsAllocated} Parts Left; {overflowBinsNeeded} quantity of {binType} Bin Needed;"

    df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = actualBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = overflowBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = overflowComment
           

# ~ 11 mins 28 sec

In [ ]:
df_Main

In [ ]:

# * Write  the Final Dataset into Excel
df_Main.to_excel('Final_Dataset.xlsx', index=False) 
# * Write  the Final BINS Dataset into Excel
df_binData.to_excel('All_Bins_Data.xlsx', index=False)

In [ ]:
wb = openpyxl.load_workbook('Final_Dataset.xlsx')
ws = wb.active

# Adjust column widths
ws.column_dimensions['A'].width = 16
ws.column_dimensions['B'].width = 52
ws.column_dimensions['C'].width = 13
ws.column_dimensions['D'].width = 0
ws.column_dimensions['E'].width = 7
ws.column_dimensions['F'].width = 15
ws.column_dimensions['G'].width = 12
ws.column_dimensions['H'].width = 12
ws.column_dimensions['I'].width = 13
ws.column_dimensions['J'].width = 11
ws.column_dimensions['K'].width = 12
ws.column_dimensions['L'].width = 9
ws.column_dimensions['M'].width = 9
ws.column_dimensions['N'].width = 9
ws.column_dimensions['O'].width = 10
ws.column_dimensions['P'].width = 24
ws.column_dimensions['Q'].width = 34
ws.column_dimensions['R'].width = 12
ws.column_dimensions['S'].width = 18
ws.column_dimensions['T'].width = 33
ws.column_dimensions['U'].width = 32
ws.column_dimensions['V'].width = 46
ws.column_dimensions['W'].width = 12

ws.column_dimensions['D'].hidden = True

ws.freeze_panes = 'A2'

for cell in ws['B']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['E']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['O']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['P']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['Q']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['R']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['S']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['T']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['U']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['V']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')


wb.save('Final_Dataset.xlsx')

In [ ]:
# ## * Main Loop for All Parts 
# for pn in df_Main['Part#']:
#     # Get & Set Variables
#     actualBin =  ""
#     overflowBin = ""
#     overflowComment = ""

#     partData = df_Main[df_Main['Part#'] == pn]
#     partHeight = partData['Height'].values[0]
#     partWidth = partData['Width'].values[0]
#     partDepth = partData['Depth'].values[0]
#     partVolume = partHeight * partWidth * partDepth 
#     binType = partData['Bin Type'].values[0]
#     partOHInven = partData['OH Inventory'].values[0]
#     totalPartVolume = partOHInven * partVolume

#     # ~ Base Continue Case - If no Storage Assignment, no Inventory Parts or is Specialty Part
#     if (binType == "") or (any([binLabelTypes in binType.lower() for binLabelTypes in ['br', 'tr', 'hs', 'bc', 'bh']])) or (partOHInven == 0):     
#         continue

#     binData = df_binData.loc[df_binData['Bin Label'] == binType]
#     totalBinOfType = binData['Total Bins'].values[0]
#     filledAmtOfBin = binData['Filled Amount'].values[0]
#     binOrder = binData['Bin Order'].values[0]

#     remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
#     partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

#     if partsAllocated > 0:   # * If Actual Bin is Availiable
#         # Calculate Variables
#         binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
#         actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
#         numBins = round(totalPartVolume / binVolume, 3)
        
#         # Set Values
#         df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
#         actualBin = f"{binType} ({actualBinRequired}, {partsAllocated})"

#         if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): # Update Availiability Flag, if full
#             df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

#         if filledAmtOfBin + numBins > totalBinOfType:   # * If Overflow (Actual Bin can't fit all Parts) (Only 1)
#             overflowParts = partOHInven - partsAllocated
            
#             # Find next suitable availiable bin to fill overflow parts
#             for newBinType in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:   
#                 if (df_binData[df_binData['Bin Label'] == newBinType]['Availiability Flag'].values[0] == 'Yes'):
#                     binData = df_binData[df_binData['Bin Label'] == newBinType]
#                     break

#             if (binType == binData['Bin Label'].values[0]): # If no bins found
#                 df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part1 - No More Available/Fitting Bins"
#                 df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
#                 continue

#             # Get & Calculate Variables
#             binType = binData['Bin Label'].values[0]
#             totalBinOfType = binData['Total Bins'].values[0]
#             filledAmtOfBin = binData['Filled Amount'].values[0]
            
#             binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
#             remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
#             partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
#             actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)

#             # Set Values -- Add Filled Amount & Build overflowBin string
#             df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
#             overflowBin = f"{binType} ({actualBinRequired}, {partsAllocated})"
        
#             # If more overflow
#             leftParts = overflowParts - partsAllocated
#             if leftParts > 0:
#                 binsNeeded = round(leftParts * partVolume / binVolume, 3)
#                 overflowComment = f"More Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"
            
#             if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): # Update Availiability Flag, if full
#                 df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

#     else:   # * If Main Bin not Availiable - Pick next suitable availiable Bin
#         for newBinType in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
#             if (df_binData[df_binData['Bin Label'] == newBinType]['Availiability Flag'].values[0] == 'Yes'):
#                 binData = df_binData[df_binData['Bin Label'] == newBinType]
#                 #  & (df_binData[df_binData['Bin Label'] == binType]["Bin Label"][1:3].isdigit())
#                 break
                
#         if (binType == binData['Bin Label'].values[0]):
#             df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = "Part2 - No More Available/Fitting Bins"
#             df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = "No More Available/Fitting Bins"
#             df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{partOHInven} Parts Left; No Bins availiable to fit"
#             continue

#         binType = binData['Bin Label'].values[0]
#         totalBinOfType = binData['Total Bins'].values[0]
#         filledAmtOfBin = binData['Filled Amount'].values[0]
#         binOrder = binData['Bin Order'].values[0]
#         binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
#         remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume   ## Check for Remaining Vol in Bin 
#         partVolume = partHeight * partWidth * partDepth 
#         totalPartVolume = partOHInven * partVolume
#         numBins = round(totalPartVolume / binVolume, 2)
#         partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)
#         actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
        
#         df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
#         actualBin = f"{binType} ({actualBinRequired}, {partsAllocated})"

#         if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): # Update Availiability Flag, if full
#             df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

#         # * If Overflow (Actual Bin can't fit all Parts) (Only 1)
#         if filledAmtOfBin + numBins > totalBinOfType:
#             overflowParts = partOHInven - partsAllocated
#             for binType in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
#                 if (df_binData[df_binData['Bin Label'] == binType]['Availiability Flag'].values[0] == 'Yes'):
#                     binData = df_binData[df_binData['Bin Label'] == binType]
#                     break

#             if (binType == binData['Bin Label'].values[0]):
#                 df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part3- No More Available/Fitting Bins"
#                 df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
#                 continue

#             binType = binData['Bin Label'].values[0]
#             binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
#             totalBinOfType = binData['Total Bins'].values[0]
#             filledAmtOfBin = binData['Filled Amount'].values[0]
#             remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
#             partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
#             actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
            
#             # Set Values -- Add Filled Amount & Build overflowBin string
#             df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
#             overflowBin = f"{binType} ({actualBinRequired}, {partsAllocated})"
        
#             leftParts = overflowParts - partsAllocated
#             if leftParts > 0:
#                 binsNeeded = round(leftParts * partVolume / binVolume, 2)
#                 overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"

#             if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): 
#                 df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

#     df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = actualBin
#     df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = overflowBin
#     df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = overflowComment

# ~ 20 sec